In [1]:
from msda import preprocessing as pr
import pandas as pd

## As a first step preprocess the data to correct gene names, remove reverse reads, trembl and correct or remove outdated identifiers

In [2]:
# NOTE. All excel and .csv files need to be present in appropriate synapse folders as well.
df = pr.preprocess_dataset('../data/raw/protein_quant_20813 sum.tsv')

/Users/skartik/git_repos/msda/msda/preprocessing.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  for pr in df.Uniprot_Id.tolist()]


In [3]:
#Save processed file so that above step need not be repeated each time.
df.to_csv('../data/raw/batch8.csv', index=False) 

## First normalize all 8 batches to their respective bridge sample

In [3]:
import process_ms_data as pmd # Specific to this repo

In [4]:
# path to metadata file
metadata_filename = '../data/metadata.csv'
dfm = pd.read_csv(metadata_filename)

## Normalize each batch to corresponding bridge

In [5]:
# NOTE: all file paths below should be replaced by sypnapse ids 
df1 = pmd.read_norm('../data/raw/batch1.csv', metadata_filename, 'batch1')
df2 = pmd.read_norm('../data/raw/batch2.csv', metadata_filename, 'batch2')
df3 = pmd.read_norm('../data/raw/batch3.csv', metadata_filename, 'batch3')
df4 = pmd.read_norm('../data/raw/batch4.csv', metadata_filename, 'batch4')
df5 = pmd.read_norm('../data/raw/batch5.csv', metadata_filename, 'batch5')
df6 = pmd.read_norm('../data/raw/batch6.csv', metadata_filename, 'batch6')
df7 = pmd.read_norm('../data/raw/batch7.csv', metadata_filename, 'batch7')
df8 = pmd.read_norm('../data/raw/batch8.csv', metadata_filename, 'batch8')

## Next normalize across all batches using a reference batch i.e Batch4

In [6]:
dfl = [df1, df2, df3, df4, df5, df6, df7, df8]
dfn = []
for i, df in enumerate(dfl):
    batch_name = 'batch%s' % str(i+1)
    samples = dfm[dfm.batch == batch_name]['sample'].tolist()
    dfn.append(pmd.normalize_between_batches(df, df4, samples))

In [7]:
dfs = pd.concat(dfn, axis=1)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [9]:
dfs.to_csv('8_batches_mergedv.csv')
